In [1]:
import numpy as np
import openai
import pandas as pd
import json
from google.cloud import secretmanager
from google.cloud import storage
import faiss
from string import Template

In [2]:
def access_secret(secret_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/1052202528756/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

In [3]:
openai_client = openai.OpenAI(api_key = access_secret("OPENAI_API_KEY"))

In [4]:
# Create dataframe of information
with open("/Users/openstockltd/Documents/Chatbot/V2/Preprocessing V2/olivia_information_vectors.json") as f:
    data = json.load(f)

hyperparams = data["hyperparams"]

embedding_model = hyperparams["embedding_model"]
chat_model = hyperparams["chat_model"]

In [5]:
vector_db = pd.DataFrame(data['data'])

vector_db

print(len(vector_db.iloc[0]['vector_embedding']))

KeyError: 'data'

## Embedding and FAISS Vector Search Handling

In [ ]:
# Create matrix of information vectors
embedding_matrix = np.vstack(vector_db['vector_embedding'].values).astype('float32')

# Generate FAISS index
dim = embedding_matrix.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embedding_matrix)

In [ ]:
def get_embedded_vector(text):
    '''
    Returns an array of the text embedded in vector space by chosen OpenAI model
    '''
    try:
        vector = openai_client.embeddings.create(input=text, model=embedding_model).data[0].embedding
        return np.array([vector], dtype=np.float32)
    except Exception as e:
        print(f"Embedding generation failed: {e}")
        raise

def knn_search(embedding, index, k, p_cosine_min):
    """
    Inputs:
    embedding: numpy array of embedded text
    index: FAISS index
    p_cosine_min: minimum threshold for 'near' neighbours
    K: number of nearest neighbours to find

    Returns:
    indices: an array of indices of near enough neighbours (in descending order by cosine similarity)
    """

    cosines, indices = index.search(embedding, k)

    return [idx for idx, sim in zip(indices[0], cosines[0]) if sim >= p_cosine_min]

## Single Input

In [ ]:
user_input = "receipt printers."

embedded_vector = get_embedded_vector(user_input)


print(knn_search(embedded_vector, index, 4, 0.3))

In [ ]:
print(confidence)

print(vector_db.iloc[indices[0]]['information_en'])

In [ ]:
# 0.3 seems to be a good lowest confidence level, possible lower

## Full loop for testing

## Summary rewriter

In [ ]:
message_history = [
    {"role": "user", "content": "Hi, I'm having trouble logging in."},
    {"role": "assistant", "content": "Have you tried resetting your password?"},
    {"role": "user", "content": "Yes, but I never got the email."}
]

# Flatten them into a string
flattened = "\n".join([f"{m['role']}: {m['content']}" for m in message_history])

print(flattened)

In [ ]:
prompt = "You are updating a short summary of a customer support conversation.\nEnsure that you capture:\n- Main device/product/service\n- Main problem or request\n- Key details already provided (OS, model, network, etc.)\n- Actions already tried\n\nUpdate the summary with any new, relevant info from the following (may be blank if summary not yet generated):\nCurrent summary:\n{summary}\n\nLast {num_turns} turns:\n{flattened_message_block}\n\nReturn only the short, updated summary."

print(prompt)

final_prompt = prompt.format(
    summary="blah blah summary",
    num_turns=5555,
    flattened_message_block="user: this adoujguhsdijgpisd, assistant: soidgjosijgposgpojsd"
)

print(final_prompt)